In [9]:
import pandas as pd

# 데이터 준비
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

# 데이터 탐색
# train.info()
# test.info()

# print(train.isnull().sum())
# print(test.isnull().sum())

# 결측치 대체(문자형 자료) - 최빈값
train['workclass'].fillna(train['workclass'].mode()[0], inplace=True)
train['native.country'].fillna(train['native.country'].mode()[0], inplace=True)
train['occupation'].fillna(train['occupation'].mode()[0], inplace=True)

# test 대체
test['workclass'].fillna(test['workclass'].mode()[0], inplace=True)
test['native.country'].fillna(test['native.country'].mode()[0], inplace=True)
test['occupation'].fillna(test['occupation'].mode()[0], inplace=True)

# 결측치 대체 - 수치형 자료(중앙값)
train['age'].fillna(train['age'].median(), inplace=True)
train['hours.per.week'].fillna(train['hours.per.week'].median(), inplace=True)

test['age'].fillna(test['age'].median(), inplace=True)
test['hours.per.week'].fillna(test['hours.per.week'].median(), inplace=True)

# print(train.isnull().sum())
# print(test.isnull().sum())

# 검증용 데이터 분할을 위한 작업
# 데이터 선택(X, y)
# 훈련용 칼럼(독립변수)
# train에 있던 income 칼럼을 삭제해서 y_train에 넣음
y_train = train.pop("income")

cols = ["workclass", "education", "marital.status", "occupation", 
        "relationship", "race", "sex", "native.country"]

# sklearn(사이킷런) - preprocessing, model_selection
# 인코딩(encoding) - 원핫 인코딩, 라벨 인코딩
# 문자데이터는 넣을 수 없으므로 숫자로 변환
"""
from sklearn.preprocessing import LabelEncoder

for col in cols:
  le = LabelEncoder()
  train[col] = le.fit_transform(train[col])
  test[col] = le.fit_transform(test[col])
"""

# one-hot 인코딩
print(train.shape, test.shape)

# 원-핫 인코딩 
# train = pd.get_dummies(train)
# test = pd.get_dummies(test)
# print(train.shape, test.shape)

# 칼럼수가 일치하지 않아 사용할 수 없으므로 데이터를 합쳐야 함
data = pd.concat([train, test], axis=0)
data = pd.get_dummies(data)
train = data.iloc[:len(train)].copy()
test = data.iloc[len(train):].copy()
print(train.shape, test.shape)

# min-max 스캐일링
from sklearn.preprocessing import MinMaxScaler

# 수치형 컬럼명
cols = ['age', 'fnlwgt', 'education.num', 'capital.gain', 'capital.loss', 'hours.per.week']

scaler = MinMaxScaler()
# display(train[cols].head(2))
train[cols] = scaler.fit_transform(train[cols])
test[cols] = scaler.transform(test[cols])
# display(train[cols].head(2))

train[cols].head(3)

# print(train.info())
# print(test.info())
# print(y_train.value_counts())

# 검증 데이터 나누기(훈련:80, 검증: 20)
from sklearn.model_selection import train_test_split

# help(train_test_split)
X_train, X_test, y_train, y_test = train_test_split(train, 
                                                    y_train, 
                                                    test_size=0.2, 
                                                    random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# 머신러닝 학습 및 평가
# 의사결정나무(Decision Tree) -> 랜덤 포레스트(RandomForest)
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
pred = rf.predict_proba(X_test)
# print(pred)

# 성능 평가 - 검증용 실제값과 예측값 비교
from sklearn.metrics import roc_auc_score
# >50K인 열을 지정[: 1]
score = roc_auc_score(y_test, pred[:, 1])
print("score:", score)

# test 자료를 검증(최종)
pred2 = rf.predict_proba(test)
# print(pred2)

# 파일 생성 및 제출
submit = pd.DataFrame({
    "pred": pred2[:, 1]
})

submit.to_csv("result.csv", index=False)

pd.read_csv("result.csv")

C:\Users\LG\AppData\Local\Temp\ipykernel_10136\3139217191.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train['workclass'].fillna(train['workclass'].mode()[0], inplace=True)
C:\Users\LG\AppData\Local\Temp\ipykernel_10136\3139217191.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as

(29304, 15) (3257, 15)
(29304, 106) (3257, 106)
(23443, 106) (5861, 106) (23443,) (5861,)
score: 0.9078019733431076


,pred
0,0.02
1,0.03
2,0.13
3,0.79
4,0.09
...,...
3252,0.00
3253,0.50
3254,0.06
3255,0.00
